# Preprocessing oldlisting_buy_2

In [1]:
import sys, os
sys.path.append(os.path.abspath('../'))
from scripts.utils import create_dir, get_runtime
import time 
start_time = time.time()

import pandas as pd
import re
import ast

In [2]:
# Load your dataset (replace with the correct path to your CSV file)
file_path = "../data/landing/oldlistings_buy/oldlistings_buy_2.csv"
df = pd.read_csv(file_path)

# Helper functions to extract and process data
def expand_rented_prices(row):
    try:
        rent_list = ast.literal_eval(row['rented_prices'])
        rows = []
        for rent in rent_list:
            new_row = row.copy()
            new_row['rented_price'] = rent.get('price', None)
            new_row['date'] = rent.get('date', None)
            rows.append(new_row)
        return pd.DataFrame(rows)
    except (ValueError, SyntaxError):
        return pd.DataFrame([row])

def extract_from_meta_data(meta_data_str, label):
    try:
        meta_list = ast.literal_eval(meta_data_str)
        for item in meta_list:
            if item.get('label') == label:
                return item.get('quantity', None)
    except Exception:
        return None

# Apply the process to expand the first 100 rows (or all rows if needed)
expanded_rows = pd.concat([expand_rented_prices(row) for _, row in df.iterrows()], ignore_index=True)

# Extract meta_data columns for bed, bath, car, land, type
expanded_rows['bed'] = expanded_rows['meta_data'].apply(lambda x: extract_from_meta_data(x, 'bed'))
expanded_rows['bath'] = expanded_rows['meta_data'].apply(lambda x: extract_from_meta_data(x, 'bath'))
expanded_rows['car'] = expanded_rows['meta_data'].apply(lambda x: extract_from_meta_data(x, 'car'))
expanded_rows['land'] = expanded_rows['meta_data'].apply(lambda x: extract_from_meta_data(x, 'land'))
expanded_rows['type'] = expanded_rows['meta_data'].apply(lambda x: extract_from_meta_data(x, 'type'))

# Keep only relevant columns
final_expanded_df = expanded_rows[['lat', 'lng', 'address', 'bed', 'bath', 'car', 'land', 'type', 'rented_price', 'date']]

# Optionally, print or view the dataframe
# print(final_expanded_df)

In [3]:
# Function to clean rented_prices
final_expanded_df['property_price_cleaned'] = (
    final_expanded_df['rented_price']
    .str.replace(r'\u200b', '', regex=False)           # Remove zero-width space
    .str.replace(r'\xa0', '', regex=False)             # Remove non-breaking space
    .str.replace(r'<span>', '', regex=False)           # Remove <span> tag
    .str.replace(r'</span>', '', regex=False)          # Remove </span> tag
    #.str.translate(translation_table)                 # Convert full-width digits to ASCII digits
    # Convert full-width digits to ASCII digits
    .str.replace('０', '0')
    .str.replace('１', '1')
    .str.replace('２', '2')
    .str.replace('３', '3')
    .str.replace('４', '4')
    .str.replace('５', '5')
    .str.replace('６', '6')
    .str.replace('７', '7')
    .str.replace('８', '8')
    .str.replace('９', '9')
    .str.replace('4', '4')
    .str.replace('𝟻', '5')
    .str.replace('𝟼', '6')
    .str.replace('𝟽', '7')
    .str.replace('𝟾', '8')
    .str.replace('𝟿', '9')
    .str.replace('𝟶', '0')
    .str.replace('𝟹', '3')
    .str.replace('𝟺', '4')
    .str.replace('𝟸', '2')
    .str.replace('𝟷', '1')
    .str.replace('𝟫', '9')
    .str.replace('𝟪', '8')
    .str.replace('𝟩', '7')
    .str.replace('𝟨', '6')
    .str.replace('𝟧', '5')
    .str.replace('𝟦', '4')
    .str.replace('𝟥', '3')
    .str.replace('𝟤', '2')
    .str.replace('𝟣', '1')
    .str.replace('𝟢', '0')
    .str.replace('𝟡', '9')
    .str.replace('𝟠', '8')
    .str.replace('𝟟', '7')
    .str.replace('𝟞', '6')
    .str.replace('𝟝', '5')
    .str.replace('𝟜', '4')
    .str.replace('𝟛', '3')
    .str.replace('𝟚', '2')
    .str.replace('𝟙', '1')
    .str.replace('𝟘', '0')
    .str.replace('𝟗', '9')
    .str.replace('𝟖', '8')
    .str.replace('𝟕', '7')
    .str.replace('𝟔', '6')
    .str.replace('𝟓', '5')
    .str.replace('𝟒', '4')
    .str.replace('𝟑', '3')
    .str.replace('𝟐', '2')
    .str.replace('𝟏', '1')
    .str.replace('𝟎', '0')
    .str.replace('𝟌', '4')
    .str.replace('𝟋', '3')
    .str.replace('𝟊', '2')
    .str.replace('𝟉', '1')
    .str.replace('𝟈', '0')
    .str.replace('𝟇', '9')
    .str.replace('𝟆', '8')
    .str.replace('𝟅', '7')
    .str.replace('𝟄', '6')
    .str.replace('𝟃', '5')
    .str.replace('𝟂', '4')
    .str.replace('𝟁', '3')
    .str.replace('𝟀', '2')
    .str.replace('𝞿', '1')
    .str.replace('𝞾', '0')
)

/tmp/ipykernel_10287/2278130057.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_expanded_df['property_price_cleaned'] = (


In [4]:
# Replace "O" with "0" in rented_price_cleaned
final_expanded_df['property_price_cleaned'] = final_expanded_df['property_price_cleaned'].str.replace('O', '0')
final_expanded_df

/tmp/ipykernel_10287/825901540.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_expanded_df['property_price_cleaned'] = final_expanded_df['property_price_cleaned'].str.replace('O', '0')


,lat,lng,address,bed,bath,car,land,type,rented_price,date,property_price_cleaned
0,NaN,NaN,"21 CAMBRIDGE ROAD, MОՍNT MART​HA",,𝟸,𝟸,None,House,"$𝟨OO,OOO Plսs",December 2012,"$600,000 Plսs"
1,NaN,NaN,"11 BIRDROCK AVENUE, MOUNT MARTHA",4,3,2,None,House,"$9\u200b50,000 Plսs\xa0",September 2012,"$950,000 Plսs"
2,NaN,NaN,"31 HARRAP ROAD, MOUNT MARTHA",,2,2,None,House,"$520,000 Neg",October 2012,"$520,000 Neg"
3,NaN,NaN,"31 HARRAP ROAD, MOUNT MARTHA",,2,2,None,House,"$500,000",August 2011,"$500,000"
4,NaN,NaN,"31 HARRAP ROAD, MOUNT MARTHA",,2,2,None,House,"$500,000",September 2011,"$500,000"
...,...,...,...,...,...,...,...,...,...,...,...
518622,NaN,NaN,"50 FIFTH AVENUE, СHΕ𝙻SΕΑ HΕIԌHTS",3,,𝟸,643\xa0m2,None,"$560,000 Buyers Over",May 2016,"$560,000 Buyers 0ver"
518623,NaN,NaN,"50 FIFTH AVENUE, СHΕ𝙻SΕΑ HΕIԌHTS",3,,𝟸,643\xa0m2,None,"$560,000 Buyers Over",April 2016,"$560,000 Buyers 0ver"
518624,NaN,NaN,"50 FIFTH AVENUE, СHΕ𝙻SΕΑ HΕIԌHTS",3,,𝟸,643\xa0m2,None,"$400,000",June 2010,"$400,000"
518625,NaN,NaN,"４𝟸 СHΕ𝙻SΕΑ PA​RK DRI​VE, CHELSEA HEIGHTS",3,2,None,530\xa0m2,None,NaN,NaN,NaN


In [5]:
# Replace all non-numeric characters with NaN from rented_price_cleaned except for commas and "$" signs and "-" signs
final_expanded_df['property_price_cleaned'] = final_expanded_df['property_price_cleaned'].str.replace(r'[^0-9,$-]', '', regex=True)
final_expanded_df

/tmp/ipykernel_10287/3598943794.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_expanded_df['property_price_cleaned'] = final_expanded_df['property_price_cleaned'].str.replace(r'[^0-9,$-]', '', regex=True)


,lat,lng,address,bed,bath,car,land,type,rented_price,date,property_price_cleaned
0,NaN,NaN,"21 CAMBRIDGE ROAD, MОՍNT MART​HA",,𝟸,𝟸,None,House,"$𝟨OO,OOO Plսs",December 2012,"$600,000"
1,NaN,NaN,"11 BIRDROCK AVENUE, MOUNT MARTHA",4,3,2,None,House,"$9\u200b50,000 Plսs\xa0",September 2012,"$950,000"
2,NaN,NaN,"31 HARRAP ROAD, MOUNT MARTHA",,2,2,None,House,"$520,000 Neg",October 2012,"$520,000"
3,NaN,NaN,"31 HARRAP ROAD, MOUNT MARTHA",,2,2,None,House,"$500,000",August 2011,"$500,000"
4,NaN,NaN,"31 HARRAP ROAD, MOUNT MARTHA",,2,2,None,House,"$500,000",September 2011,"$500,000"
...,...,...,...,...,...,...,...,...,...,...,...
518622,NaN,NaN,"50 FIFTH AVENUE, СHΕ𝙻SΕΑ HΕIԌHTS",3,,𝟸,643\xa0m2,None,"$560,000 Buyers Over",May 2016,"$560,0000"
518623,NaN,NaN,"50 FIFTH AVENUE, СHΕ𝙻SΕΑ HΕIԌHTS",3,,𝟸,643\xa0m2,None,"$560,000 Buyers Over",April 2016,"$560,0000"
518624,NaN,NaN,"50 FIFTH AVENUE, СHΕ𝙻SΕΑ HΕIԌHTS",3,,𝟸,643\xa0m2,None,"$400,000",June 2010,"$400,000"
518625,NaN,NaN,"４𝟸 СHΕ𝙻SΕΑ PA​RK DRI​VE, CHELSEA HEIGHTS",3,2,None,530\xa0m2,None,NaN,NaN,NaN


In [6]:
# Function to clean and handle range prices by calculating the average
def clean_price(price):
    if pd.isna(price):
        return price  # Return NaN as is
    # Handle price ranges like "$425,000-$455,000"
    range_match = re.match(r"\$(\d+,\d+)-\$(\d+,\d+)", price)
    if range_match:
        low_price = int(range_match.group(1).replace(',', ''))
        high_price = int(range_match.group(2).replace(',', ''))
        return (low_price + high_price) / 2  # Return the average of the range
    # Handle normal prices
    price_cleaned = re.sub(r'[^\d]', '', price)
    return int(price_cleaned) if price_cleaned.isdigit() else None

In [7]:
final_expanded_df['property_price_cleaned'] = [clean_price(price) for price in final_expanded_df['property_price_cleaned']]

final_expanded_df

/tmp/ipykernel_10287/2455915571.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_expanded_df['property_price_cleaned'] = [clean_price(price) for price in final_expanded_df['property_price_cleaned']]


,lat,lng,address,bed,bath,car,land,type,rented_price,date,property_price_cleaned
0,NaN,NaN,"21 CAMBRIDGE ROAD, MОՍNT MART​HA",,𝟸,𝟸,None,House,"$𝟨OO,OOO Plսs",December 2012,600000.0
1,NaN,NaN,"11 BIRDROCK AVENUE, MOUNT MARTHA",4,3,2,None,House,"$9\u200b50,000 Plսs\xa0",September 2012,950000.0
2,NaN,NaN,"31 HARRAP ROAD, MOUNT MARTHA",,2,2,None,House,"$520,000 Neg",October 2012,520000.0
3,NaN,NaN,"31 HARRAP ROAD, MOUNT MARTHA",,2,2,None,House,"$500,000",August 2011,500000.0
4,NaN,NaN,"31 HARRAP ROAD, MOUNT MARTHA",,2,2,None,House,"$500,000",September 2011,500000.0
...,...,...,...,...,...,...,...,...,...,...,...
518622,NaN,NaN,"50 FIFTH AVENUE, СHΕ𝙻SΕΑ HΕIԌHTS",3,,𝟸,643\xa0m2,None,"$560,000 Buyers Over",May 2016,5600000.0
518623,NaN,NaN,"50 FIFTH AVENUE, СHΕ𝙻SΕΑ HΕIԌHTS",3,,𝟸,643\xa0m2,None,"$560,000 Buyers Over",April 2016,5600000.0
518624,NaN,NaN,"50 FIFTH AVENUE, СHΕ𝙻SΕΑ HΕIԌHTS",3,,𝟸,643\xa0m2,None,"$400,000",June 2010,400000.0
518625,NaN,NaN,"４𝟸 СHΕ𝙻SΕΑ PA​RK DRI​VE, CHELSEA HEIGHTS",3,2,None,530\xa0m2,None,NaN,NaN,NaN


In [8]:
final_expanded_df['date'] = pd.to_datetime(final_expanded_df['date'], errors='coerce')
final_expanded_df

/tmp/ipykernel_10287/1390822886.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  final_expanded_df['date'] = pd.to_datetime(final_expanded_df['date'], errors='coerce')
/tmp/ipykernel_10287/1390822886.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_expanded_df['date'] = pd.to_datetime(final_expanded_df['date'], errors='coerce')


,lat,lng,address,bed,bath,car,land,type,rented_price,date,property_price_cleaned
0,NaN,NaN,"21 CAMBRIDGE ROAD, MОՍNT MART​HA",,𝟸,𝟸,None,House,"$𝟨OO,OOO Plսs",2012-12-01,600000.0
1,NaN,NaN,"11 BIRDROCK AVENUE, MOUNT MARTHA",4,3,2,None,House,"$9\u200b50,000 Plսs\xa0",2012-09-01,950000.0
2,NaN,NaN,"31 HARRAP ROAD, MOUNT MARTHA",,2,2,None,House,"$520,000 Neg",2012-10-01,520000.0
3,NaN,NaN,"31 HARRAP ROAD, MOUNT MARTHA",,2,2,None,House,"$500,000",2011-08-01,500000.0
4,NaN,NaN,"31 HARRAP ROAD, MOUNT MARTHA",,2,2,None,House,"$500,000",2011-09-01,500000.0
...,...,...,...,...,...,...,...,...,...,...,...
518622,NaN,NaN,"50 FIFTH AVENUE, СHΕ𝙻SΕΑ HΕIԌHTS",3,,𝟸,643\xa0m2,None,"$560,000 Buyers Over",2016-05-01,5600000.0
518623,NaN,NaN,"50 FIFTH AVENUE, СHΕ𝙻SΕΑ HΕIԌHTS",3,,𝟸,643\xa0m2,None,"$560,000 Buyers Over",2016-04-01,5600000.0
518624,NaN,NaN,"50 FIFTH AVENUE, СHΕ𝙻SΕΑ HΕIԌHTS",3,,𝟸,643\xa0m2,None,"$400,000",2010-06-01,400000.0
518625,NaN,NaN,"４𝟸 СHΕ𝙻SΕΑ PA​RK DRI​VE, CHELSEA HEIGHTS",3,2,None,530\xa0m2,None,NaN,NaT,NaN


In [9]:
# Function to clean "bed"
final_expanded_df['bed_cleaned'] = (
    final_expanded_df['bed']
    .str.replace(r'\u200b', '', regex=False)           # Remove zero-width space
    .str.replace(r'\xa0', '', regex=False)             # Remove non-breaking space
    .str.replace(r'<span>', '', regex=False)           # Remove <span> tag
    .str.replace(r'</span>', '', regex=False)          # Remove </span> tag
    #.str.translate(translation_table)                 # Convert full-width digits to ASCII digits
    # Convert full-width digits to ASCII digits
    .str.replace('０', '0')
    .str.replace('１', '1')
    .str.replace('２', '2')
    .str.replace('３', '3')
    .str.replace('４', '4')
    .str.replace('５', '5')
    .str.replace('６', '6')
    .str.replace('７', '7')
    .str.replace('８', '8')
    .str.replace('９', '9')
    .str.replace('4', '4')
    .str.replace('𝟻', '5')
    .str.replace('𝟼', '6')
    .str.replace('𝟽', '7')
    .str.replace('𝟾', '8')
    .str.replace('𝟿', '9')
    .str.replace('𝟶', '0')
    .str.replace('𝟹', '3')
    .str.replace('𝟺', '4')
    .str.replace('𝟸', '2')
    .str.replace('𝟷', '1')
    .str.replace('𝟫', '9')
    .str.replace('𝟪', '8')
    .str.replace('𝟩', '7')
    .str.replace('𝟨', '6')
    .str.replace('𝟧', '5')
    .str.replace('𝟦', '4')
    .str.replace('𝟥', '3')
    .str.replace('𝟤', '2')
    .str.replace('𝟣', '1')
    .str.replace('𝟢', '0')
    .str.replace('𝟡', '9')
    .str.replace('𝟠', '8')
    .str.replace('𝟟', '7')
    .str.replace('𝟞', '6')
    .str.replace('𝟝', '5')
    .str.replace('𝟜', '4')
    .str.replace('𝟛', '3')
    .str.replace('𝟚', '2')
    .str.replace('𝟙', '1')
    .str.replace('𝟘', '0')
    .str.replace('𝟗', '9')
    .str.replace('𝟖', '8')
    .str.replace('𝟕', '7')
    .str.replace('𝟔', '6')
    .str.replace('𝟓', '5')
    .str.replace('𝟒', '4')
    .str.replace('𝟑', '3')
    .str.replace('𝟐', '2')
    .str.replace('𝟏', '1')
    .str.replace('𝟎', '0')
    .str.replace('𝟌', '4')
    .str.replace('𝟋', '3')
    .str.replace('𝟊', '2')
    .str.replace('𝟉', '1')
    .str.replace('𝟈', '0')
    .str.replace('𝟇', '9')
    .str.replace('𝟆', '8')
    .str.replace('𝟅', '7')
    .str.replace('𝟄', '6')
    .str.replace('𝟃', '5')
    .str.replace('𝟂', '4')
    .str.replace('𝟁', '3')
    .str.replace('𝟀', '2')
    .str.replace('𝞿', '1')
    .str.replace('𝞾', '0')
)

/tmp/ipykernel_10287/2184586359.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_expanded_df['bed_cleaned'] = (


In [10]:
# Function to clean "bath"
final_expanded_df['bath_cleaned'] = (
    final_expanded_df['bath']
    .str.replace(r'\u200b', '', regex=False)           # Remove zero-width space
    .str.replace(r'\xa0', '', regex=False)             # Remove non-breaking space
    .str.replace(r'<span>', '', regex=False)           # Remove <span> tag
    .str.replace(r'</span>', '', regex=False)          # Remove </span> tag
    #.str.translate(translation_table)                 # Convert full-width digits to ASCII digits
    # Convert full-width digits to ASCII digits
    .str.replace('０', '0')
    .str.replace('１', '1')
    .str.replace('２', '2')
    .str.replace('３', '3')
    .str.replace('４', '4')
    .str.replace('５', '5')
    .str.replace('６', '6')
    .str.replace('７', '7')
    .str.replace('８', '8')
    .str.replace('９', '9')
    .str.replace('4', '4')
    .str.replace('𝟻', '5')
    .str.replace('𝟼', '6')
    .str.replace('𝟽', '7')
    .str.replace('𝟾', '8')
    .str.replace('𝟿', '9')
    .str.replace('𝟶', '0')
    .str.replace('𝟹', '3')
    .str.replace('𝟺', '4')
    .str.replace('𝟸', '2')
    .str.replace('𝟷', '1')
    .str.replace('𝟫', '9')
    .str.replace('𝟪', '8')
    .str.replace('𝟩', '7')
    .str.replace('𝟨', '6')
    .str.replace('𝟧', '5')
    .str.replace('𝟦', '4')
    .str.replace('𝟥', '3')
    .str.replace('𝟤', '2')
    .str.replace('𝟣', '1')
    .str.replace('𝟢', '0')
    .str.replace('𝟡', '9')
    .str.replace('𝟠', '8')
    .str.replace('𝟟', '7')
    .str.replace('𝟞', '6')
    .str.replace('𝟝', '5')
    .str.replace('𝟜', '4')
    .str.replace('𝟛', '3')
    .str.replace('𝟚', '2')
    .str.replace('𝟙', '1')
    .str.replace('𝟘', '0')
    .str.replace('𝟗', '9')
    .str.replace('𝟖', '8')
    .str.replace('𝟕', '7')
    .str.replace('𝟔', '6')
    .str.replace('𝟓', '5')
    .str.replace('𝟒', '4')
    .str.replace('𝟑', '3')
    .str.replace('𝟐', '2')
    .str.replace('𝟏', '1')
    .str.replace('𝟎', '0')
    .str.replace('𝟌', '4')
    .str.replace('𝟋', '3')
    .str.replace('𝟊', '2')
    .str.replace('𝟉', '1')
    .str.replace('𝟈', '0')
    .str.replace('𝟇', '9')
    .str.replace('𝟆', '8')
    .str.replace('𝟅', '7')
    .str.replace('𝟄', '6')
    .str.replace('𝟃', '5')
    .str.replace('𝟂', '4')
    .str.replace('𝟁', '3')
    .str.replace('𝟀', '2')
    .str.replace('𝞿', '1')
    .str.replace('𝞾', '0')
)

In [11]:
# Function to clean "car"
final_expanded_df['car_cleaned'] = (
    final_expanded_df['car']
    .str.replace(r'\u200b', '', regex=False)           # Remove zero-width space
    .str.replace(r'\xa0', '', regex=False)             # Remove non-breaking space
    .str.replace(r'<span>', '', regex=False)           # Remove <span> tag
    .str.replace(r'</span>', '', regex=False)          # Remove </span> tag
    #.str.translate(translation_table)                 # Convert full-width digits to ASCII digits
    # Convert full-width digits to ASCII digits
    .str.replace('０', '0')
    .str.replace('１', '1')
    .str.replace('２', '2')
    .str.replace('３', '3')
    .str.replace('４', '4')
    .str.replace('５', '5')
    .str.replace('６', '6')
    .str.replace('７', '7')
    .str.replace('８', '8')
    .str.replace('９', '9')
    .str.replace('4', '4')
    .str.replace('𝟻', '5')
    .str.replace('𝟼', '6')
    .str.replace('𝟽', '7')
    .str.replace('𝟾', '8')
    .str.replace('𝟿', '9')
    .str.replace('𝟶', '0')
    .str.replace('𝟹', '3')
    .str.replace('𝟺', '4')
    .str.replace('𝟸', '2')
    .str.replace('𝟷', '1')
    .str.replace('𝟫', '9')
    .str.replace('𝟪', '8')
    .str.replace('𝟩', '7')
    .str.replace('𝟨', '6')
    .str.replace('𝟧', '5')
    .str.replace('𝟦', '4')
    .str.replace('𝟥', '3')
    .str.replace('𝟤', '2')
    .str.replace('𝟣', '1')
    .str.replace('𝟢', '0')
    .str.replace('𝟡', '9')
    .str.replace('𝟠', '8')
    .str.replace('𝟟', '7')
    .str.replace('𝟞', '6')
    .str.replace('𝟝', '5')
    .str.replace('𝟜', '4')
    .str.replace('𝟛', '3')
    .str.replace('𝟚', '2')
    .str.replace('𝟙', '1')
    .str.replace('𝟘', '0')
    .str.replace('𝟗', '9')
    .str.replace('𝟖', '8')
    .str.replace('𝟕', '7')
    .str.replace('𝟔', '6')
    .str.replace('𝟓', '5')
    .str.replace('𝟒', '4')
    .str.replace('𝟑', '3')
    .str.replace('𝟐', '2')
    .str.replace('𝟏', '1')
    .str.replace('𝟎', '0')
    .str.replace('𝟌', '4')
    .str.replace('𝟋', '3')
    .str.replace('𝟊', '2')
    .str.replace('𝟉', '1')
    .str.replace('𝟈', '0')
    .str.replace('𝟇', '9')
    .str.replace('𝟆', '8')
    .str.replace('𝟅', '7')
    .str.replace('𝟄', '6')
    .str.replace('𝟃', '5')
    .str.replace('𝟂', '4')
    .str.replace('𝟁', '3')
    .str.replace('𝟀', '2')
    .str.replace('𝞿', '1')
    .str.replace('𝞾', '0')
)

In [12]:
# Function to clean "land"
final_expanded_df['land_cleaned'] = (
    final_expanded_df['land']
    .str.replace(r'm2', '', regex=False)           # Remove "m2" unit
    .str.replace(r'\u200b', '', regex=False)           # Remove zero-width space
    .str.replace(r'\xa0', '', regex=False)             # Remove non-breaking space
    .str.replace(r'<span>', '', regex=False)           # Remove <span> tag
    .str.replace(r'</span>', '', regex=False)          # Remove </span> tag
    #.str.translate(translation_table)                 # Convert full-width digits to ASCII digits
    # Convert full-width digits to ASCII digits
    .str.replace('０', '0')
    .str.replace('１', '1')
    .str.replace('２', '2')
    .str.replace('３', '3')
    .str.replace('４', '4')
    .str.replace('５', '5')
    .str.replace('６', '6')
    .str.replace('７', '7')
    .str.replace('８', '8')
    .str.replace('９', '9')
    .str.replace('4', '4')
    .str.replace('𝟻', '5')
    .str.replace('𝟼', '6')
    .str.replace('𝟽', '7')
    .str.replace('𝟾', '8')
    .str.replace('𝟿', '9')
    .str.replace('𝟶', '0')
    .str.replace('𝟹', '3')
    .str.replace('𝟺', '4')
    .str.replace('𝟸', '2')
    .str.replace('𝟷', '1')
    .str.replace('𝟫', '9')
    .str.replace('𝟪', '8')
    .str.replace('𝟩', '7')
    .str.replace('𝟨', '6')
    .str.replace('𝟧', '5')
    .str.replace('𝟦', '4')
    .str.replace('𝟥', '3')
    .str.replace('𝟤', '2')
    .str.replace('𝟣', '1')
    .str.replace('𝟢', '0')
    .str.replace('𝟡', '9')
    .str.replace('𝟠', '8')
    .str.replace('𝟟', '7')
    .str.replace('𝟞', '6')
    .str.replace('𝟝', '5')
    .str.replace('𝟜', '4')
    .str.replace('𝟛', '3')
    .str.replace('𝟚', '2')
    .str.replace('𝟙', '1')
    .str.replace('𝟘', '0')
    .str.replace('𝟗', '9')
    .str.replace('𝟖', '8')
    .str.replace('𝟕', '7')
    .str.replace('𝟔', '6')
    .str.replace('𝟓', '5')
    .str.replace('𝟒', '4')
    .str.replace('𝟑', '3')
    .str.replace('𝟐', '2')
    .str.replace('𝟏', '1')
    .str.replace('𝟎', '0')
    .str.replace('𝟌', '4')
    .str.replace('𝟋', '3')
    .str.replace('𝟊', '2')
    .str.replace('𝟉', '1')
    .str.replace('𝟈', '0')
    .str.replace('𝟇', '9')
    .str.replace('𝟆', '8')
    .str.replace('𝟅', '7')
    .str.replace('𝟄', '6')
    .str.replace('𝟃', '5')
    .str.replace('𝟂', '4')
    .str.replace('𝟁', '3')
    .str.replace('𝟀', '2')
    .str.replace('𝞿', '1')
    .str.replace('𝞾', '0')
)

In [13]:
final_expanded_df

,lat,lng,address,bed,bath,car,land,type,rented_price,date,property_price_cleaned,bed_cleaned,bath_cleaned,car_cleaned,land_cleaned
0,NaN,NaN,"21 CAMBRIDGE ROAD, MОՍNT MART​HA",,𝟸,𝟸,None,House,"$𝟨OO,OOO Plսs",2012-12-01,600000.0,,2,2,None
1,NaN,NaN,"11 BIRDROCK AVENUE, MOUNT MARTHA",4,3,2,None,House,"$9\u200b50,000 Plսs\xa0",2012-09-01,950000.0,4,3,2,None
2,NaN,NaN,"31 HARRAP ROAD, MOUNT MARTHA",,2,2,None,House,"$520,000 Neg",2012-10-01,520000.0,,2,2,None
3,NaN,NaN,"31 HARRAP ROAD, MOUNT MARTHA",,2,2,None,House,"$500,000",2011-08-01,500000.0,,2,2,None
4,NaN,NaN,"31 HARRAP ROAD, MOUNT MARTHA",,2,2,None,House,"$500,000",2011-09-01,500000.0,,2,2,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518622,NaN,NaN,"50 FIFTH AVENUE, СHΕ𝙻SΕΑ HΕIԌHTS",3,,𝟸,643\xa0m2,None,"$560,000 Buyers Over",2016-05-01,5600000.0,3,,2,643
518623,NaN,NaN,"50 FIFTH AVENUE, СHΕ𝙻SΕΑ HΕIԌHTS",3,,𝟸,643\xa0m2,None,"$560,000 Buyers Over",2016-04-01,5600000.0,3,,2,643
518624,NaN,NaN,"50 FIFTH AVENUE, СHΕ𝙻SΕΑ HΕIԌHTS",3,,𝟸,643\xa0m2,None,"$400,000",2010-06-01,400000.0,3,,2,643
518625,NaN,NaN,"４𝟸 СHΕ𝙻SΕΑ PA​RK DRI​VE, CHELSEA HEIGHTS",3,2,None,530\xa0m2,None,NaN,NaT,NaN,3,2,None,530


In [14]:
# Function to clean "address"
final_expanded_df['address_cleaned'] = (
    final_expanded_df['address']
    .str.replace(r'\u200b', '', regex=False)           # Remove zero-width space
    .str.replace(r'\xa0', '', regex=False)             # Remove non-breaking space
    .str.replace(r'<span>', '', regex=False)           # Remove <span> tag
    .str.replace(r'</span>', '', regex=False)          # Remove </span> tag
    .str.replace('𝙻', 'L')
    .str.replace('𝙺', 'K')
    .str.replace('𝙹', 'J')
    .str.replace('𝙸', 'I')
    .str.replace('𝙷', 'H')
    .str.replace('𝙶', 'G')
    .str.replace('𝙵', 'F')
    .str.replace('𝙴', 'E')
    .str.replace('𝙳', 'D')
    .str.replace('𝙲', 'C')
    .str.replace('𝙱', 'B')
    .str.replace('𝙰', 'A')
    .str.replace('𝘾', 'C')
    .str.replace('𝘽', 'B')
    .str.replace('𝘼', 'A')
    .str.replace('𝘿', 'D')
    .str.replace('𝘾', 'C')
    .str.replace('𝘽', 'B')
    .str.replace('𝘼', 'A')
    .str.replace('𝘻', 'z')
    .str.replace('𝘺', 'y')
    .str.replace('𝘹', 'x')
    .str.replace('𝘸', 'w')
    .str.replace('𝘷', 'v')
    .str.replace('𝘶', 'u')
    .str.replace('𝘵', 't')
    .str.replace('𝘴', 's')

)

In [15]:
# Function to clean "address"
final_expanded_df['address_cleaned'] = (
    final_expanded_df['address_cleaned']
    .str.replace('𝘳', 'r')
    .str.replace('𝘲', 'q')
    .str.replace('𝘱', 'p')
    .str.replace('𝘰', 'o')
    .str.replace('𝘯', 'n')
    .str.replace('𝘭', 'l')
    .str.replace('𝘬', 'k')
    .str.replace('𝘫', 'j')
    .str.replace('𝘪', 'i')
    .str.replace('𝘩', 'h')
    .str.replace('𝘨', 'g')
    .str.replace('𝘧', 'f')
    .str.replace('𝘦', 'e')
    .str.replace('𝘥', 'd')
    .str.replace('𝘣', 'b')
    .str.replace('𝘢', 'a')
    .str.replace('𝘡', 'Z')
    .str.replace('𝘠', 'Y')
    .str.replace('𝘟', 'X')
    .str.replace('０', '0')
    .str.replace('１', '1')
    .str.replace('２', '2')
    .str.replace('３', '3')
    .str.replace('４', '4')
    .str.replace('５', '5')
    .str.replace('６', '6')
    .str.replace('７', '7')
    .str.replace('８', '8')
    .str.replace('９', '9')
    .str.replace('4', '4')
    .str.replace('𝟻', '5')
    .str.replace('𝟼', '6')
    .str.replace('𝟽', '7')
    .str.replace('𝟾', '8')
    .str.replace('𝟿', '9')
    .str.replace('𝟶', '0')
    .str.replace('𝟹', '3')
    .str.replace('𝟺', '4')
    .str.replace('𝟸', '2')
    .str.replace('𝟷', '1')
    .str.replace('𝟫', '9')
    .str.replace('𝟪', '8')
    .str.replace('𝟩', '7')
    .str.replace('𝟨', '6')
    .str.replace('𝟧', '5')
    .str.replace('𝟦', '4')
    .str.replace('𝟥', '3')
    .str.replace('𝟤', '2')
    .str.replace('𝟣', '1')
    .str.replace('𝟢', '0')
    .str.replace('𝟡', '9')
    .str.replace('𝟠', '8')
    .str.replace('𝟟', '7')
    .str.replace('𝟞', '6')
    .str.replace('𝟝', '5')
    .str.replace('𝟜', '4')
    .str.replace('𝟛', '3')
    .str.replace('𝟚', '2')
    .str.replace('𝟙', '1')
    .str.replace('𝟘', '0')
    .str.replace('𝟗', '9')
    .str.replace('𝟖', '8')
    .str.replace('𝟕', '7')
    .str.replace('𝟔', '6')
    .str.replace('𝟓', '5')
    .str.replace('𝟒', '4')
    .str.replace('𝟑', '3')
    .str.replace('𝟐', '2')
    .str.replace('𝟏', '1')
    .str.replace('𝟎', '0')
    

)

In [16]:
final_expanded_df

,lat,lng,address,bed,bath,car,land,type,rented_price,date,property_price_cleaned,bed_cleaned,bath_cleaned,car_cleaned,land_cleaned,address_cleaned
0,NaN,NaN,"21 CAMBRIDGE ROAD, MОՍNT MART​HA",,𝟸,𝟸,None,House,"$𝟨OO,OOO Plսs",2012-12-01,600000.0,,2,2,None,"21 CAMBRIDGE ROAD, MОՍNT MART​HA"
1,NaN,NaN,"11 BIRDROCK AVENUE, MOUNT MARTHA",4,3,2,None,House,"$9\u200b50,000 Plսs\xa0",2012-09-01,950000.0,4,3,2,None,"11 BIRDROCK AVENUE, MOUNT MARTHA"
2,NaN,NaN,"31 HARRAP ROAD, MOUNT MARTHA",,2,2,None,House,"$520,000 Neg",2012-10-01,520000.0,,2,2,None,"31 HARRAP ROAD, MOUNT MARTHA"
3,NaN,NaN,"31 HARRAP ROAD, MOUNT MARTHA",,2,2,None,House,"$500,000",2011-08-01,500000.0,,2,2,None,"31 HARRAP ROAD, MOUNT MARTHA"
4,NaN,NaN,"31 HARRAP ROAD, MOUNT MARTHA",,2,2,None,House,"$500,000",2011-09-01,500000.0,,2,2,None,"31 HARRAP ROAD, MOUNT MARTHA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518622,NaN,NaN,"50 FIFTH AVENUE, СHΕ𝙻SΕΑ HΕIԌHTS",3,,𝟸,643\xa0m2,None,"$560,000 Buyers Over",2016-05-01,5600000.0,3,,2,643,"50 FIFTH AVENUE, СHΕLSΕΑ HΕIԌHTS"
518623,NaN,NaN,"50 FIFTH AVENUE, СHΕ𝙻SΕΑ HΕIԌHTS",3,,𝟸,643\xa0m2,None,"$560,000 Buyers Over",2016-04-01,5600000.0,3,,2,643,"50 FIFTH AVENUE, СHΕLSΕΑ HΕIԌHTS"
518624,NaN,NaN,"50 FIFTH AVENUE, СHΕ𝙻SΕΑ HΕIԌHTS",3,,𝟸,643\xa0m2,None,"$400,000",2010-06-01,400000.0,3,,2,643,"50 FIFTH AVENUE, СHΕLSΕΑ HΕIԌHTS"
518625,NaN,NaN,"４𝟸 СHΕ𝙻SΕΑ PA​RK DRI​VE, CHELSEA HEIGHTS",3,2,None,530\xa0m2,None,NaN,NaT,NaN,3,2,None,530,"42 СHΕLSΕΑ PA​RK DRI​VE, CHELSEA HEIGHTS"


In [17]:
# drop some columns
final_expanded_df = final_expanded_df.drop(columns=['rented_price', 'bed', 'bath', 'car', 'land', 'address'])
final_expanded_df

,lat,lng,type,date,property_price_cleaned,bed_cleaned,bath_cleaned,car_cleaned,land_cleaned,address_cleaned
0,NaN,NaN,House,2012-12-01,600000.0,,2,2,None,"21 CAMBRIDGE ROAD, MОՍNT MART​HA"
1,NaN,NaN,House,2012-09-01,950000.0,4,3,2,None,"11 BIRDROCK AVENUE, MOUNT MARTHA"
2,NaN,NaN,House,2012-10-01,520000.0,,2,2,None,"31 HARRAP ROAD, MOUNT MARTHA"
3,NaN,NaN,House,2011-08-01,500000.0,,2,2,None,"31 HARRAP ROAD, MOUNT MARTHA"
4,NaN,NaN,House,2011-09-01,500000.0,,2,2,None,"31 HARRAP ROAD, MOUNT MARTHA"
...,...,...,...,...,...,...,...,...,...,...
518622,NaN,NaN,None,2016-05-01,5600000.0,3,,2,643,"50 FIFTH AVENUE, СHΕLSΕΑ HΕIԌHTS"
518623,NaN,NaN,None,2016-04-01,5600000.0,3,,2,643,"50 FIFTH AVENUE, СHΕLSΕΑ HΕIԌHTS"
518624,NaN,NaN,None,2010-06-01,400000.0,3,,2,643,"50 FIFTH AVENUE, СHΕLSΕΑ HΕIԌHTS"
518625,NaN,NaN,None,NaT,NaN,3,2,None,530,"42 СHΕLSΕΑ PA​RK DRI​VE, CHELSEA HEIGHTS"


In [18]:
# Extract year from date
final_expanded_df['year'] = final_expanded_df['date'].dt.year
final_expanded_df

,lat,lng,type,date,property_price_cleaned,bed_cleaned,bath_cleaned,car_cleaned,land_cleaned,address_cleaned,year
0,NaN,NaN,House,2012-12-01,600000.0,,2,2,None,"21 CAMBRIDGE ROAD, MОՍNT MART​HA",2012.0
1,NaN,NaN,House,2012-09-01,950000.0,4,3,2,None,"11 BIRDROCK AVENUE, MOUNT MARTHA",2012.0
2,NaN,NaN,House,2012-10-01,520000.0,,2,2,None,"31 HARRAP ROAD, MOUNT MARTHA",2012.0
3,NaN,NaN,House,2011-08-01,500000.0,,2,2,None,"31 HARRAP ROAD, MOUNT MARTHA",2011.0
4,NaN,NaN,House,2011-09-01,500000.0,,2,2,None,"31 HARRAP ROAD, MOUNT MARTHA",2011.0
...,...,...,...,...,...,...,...,...,...,...,...
518622,NaN,NaN,None,2016-05-01,5600000.0,3,,2,643,"50 FIFTH AVENUE, СHΕLSΕΑ HΕIԌHTS",2016.0
518623,NaN,NaN,None,2016-04-01,5600000.0,3,,2,643,"50 FIFTH AVENUE, СHΕLSΕΑ HΕIԌHTS",2016.0
518624,NaN,NaN,None,2010-06-01,400000.0,3,,2,643,"50 FIFTH AVENUE, СHΕLSΕΑ HΕIԌHTS",2010.0
518625,NaN,NaN,None,NaT,NaN,3,2,None,530,"42 СHΕLSΕΑ PA​RK DRI​VE, CHELSEA HEIGHTS",NaN


In [19]:
# Extract suburb from address_cleaned, which is all text after the last comma
final_expanded_df['suburb'] = final_expanded_df['address_cleaned'].str.rsplit(',').str[-1]
final_expanded_df

,lat,lng,type,date,property_price_cleaned,bed_cleaned,bath_cleaned,car_cleaned,land_cleaned,address_cleaned,year,suburb
0,NaN,NaN,House,2012-12-01,600000.0,,2,2,None,"21 CAMBRIDGE ROAD, MОՍNT MART​HA",2012.0,MОՍNT MART​HA
1,NaN,NaN,House,2012-09-01,950000.0,4,3,2,None,"11 BIRDROCK AVENUE, MOUNT MARTHA",2012.0,MOUNT MARTHA
2,NaN,NaN,House,2012-10-01,520000.0,,2,2,None,"31 HARRAP ROAD, MOUNT MARTHA",2012.0,MOUNT MARTHA
3,NaN,NaN,House,2011-08-01,500000.0,,2,2,None,"31 HARRAP ROAD, MOUNT MARTHA",2011.0,MOUNT MARTHA
4,NaN,NaN,House,2011-09-01,500000.0,,2,2,None,"31 HARRAP ROAD, MOUNT MARTHA",2011.0,MOUNT MARTHA
...,...,...,...,...,...,...,...,...,...,...,...,...
518622,NaN,NaN,None,2016-05-01,5600000.0,3,,2,643,"50 FIFTH AVENUE, СHΕLSΕΑ HΕIԌHTS",2016.0,СHΕLSΕΑ HΕIԌHTS
518623,NaN,NaN,None,2016-04-01,5600000.0,3,,2,643,"50 FIFTH AVENUE, СHΕLSΕΑ HΕIԌHTS",2016.0,СHΕLSΕΑ HΕIԌHTS
518624,NaN,NaN,None,2010-06-01,400000.0,3,,2,643,"50 FIFTH AVENUE, СHΕLSΕΑ HΕIԌHTS",2010.0,СHΕLSΕΑ HΕIԌHTS
518625,NaN,NaN,None,NaT,NaN,3,2,None,530,"42 СHΕLSΕΑ PA​RK DRI​VE, CHELSEA HEIGHTS",NaN,CHELSEA HEIGHTS


In [20]:
create_dir("../data/raw/oldlistings_buy/")
final_expanded_df.to_csv("../data/raw/oldlistings_buy/oldlistings_buy_2.csv", index=False)

Directory already exists: ../data/raw/oldlistings_buy/

